#   Video Segmentation with MedSAM2
This notebook shows how to use MedSAM2 for video segmentation inference. 

If running locally using jupyter, first install `MedSAM2` in your environment using the [installation instructions](https://github.com/bowang-lab/MedSAM2?tab=readme-ov-file#installation) in the repository.

## Load Packages and MedsSAM2 video predictor

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sam2.build_sam import build_sam2_video_predictor

# helper functions
DAVIS_PALETTE = b"\x00\x00\x00\x80\x00\x00\x00\x80\x00\x80\x80\x00\x00\x00\x80\x80\x00\x80\x00\x80\x80\x80\x80\x80@\x00\x00\xc0\x00\x00@\x80\x00\xc0\x80\x00@\x00\x80\xc0\x00\x80@\x80\x80\xc0\x80\x80\x00@\x00\x80@\x00\x00\xc0\x00\x80\xc0\x00\x00@\x80\x80@\x80\x00\xc0\x80\x80\xc0\x80@@\x00\xc0@\x00@\xc0\x00\xc0\xc0\x00@@\x80\xc0@\x80@\xc0\x80\xc0\xc0\x80\x00\x00@\x80\x00@\x00\x80@\x80\x80@\x00\x00\xc0\x80\x00\xc0\x00\x80\xc0\x80\x80\xc0@\x00@\xc0\x00@@\x80@\xc0\x80@@\x00\xc0\xc0\x00\xc0@\x80\xc0\xc0\x80\xc0\x00@@\x80@@\x00\xc0@\x80\xc0@\x00@\xc0\x80@\xc0\x00\xc0\xc0\x80\xc0\xc0@@@\xc0@@@\xc0@\xc0\xc0@@@\xc0\xc0@\xc0@\xc0\xc0\xc0\xc0\xc0 \x00\x00\xa0\x00\x00 \x80\x00\xa0\x80\x00 \x00\x80\xa0\x00\x80 \x80\x80\xa0\x80\x80`\x00\x00\xe0\x00\x00`\x80\x00\xe0\x80\x00`\x00\x80\xe0\x00\x80`\x80\x80\xe0\x80\x80 @\x00\xa0@\x00 \xc0\x00\xa0\xc0\x00 @\x80\xa0@\x80 \xc0\x80\xa0\xc0\x80`@\x00\xe0@\x00`\xc0\x00\xe0\xc0\x00`@\x80\xe0@\x80`\xc0\x80\xe0\xc0\x80 \x00@\xa0\x00@ \x80@\xa0\x80@ \x00\xc0\xa0\x00\xc0 \x80\xc0\xa0\x80\xc0`\x00@\xe0\x00@`\x80@\xe0\x80@`\x00\xc0\xe0\x00\xc0`\x80\xc0\xe0\x80\xc0 @@\xa0@@ \xc0@\xa0\xc0@ @\xc0\xa0@\xc0 \xc0\xc0\xa0\xc0\xc0`@@\xe0@@`\xc0@\xe0\xc0@`@\xc0\xe0@\xc0`\xc0\xc0\xe0\xc0\xc0\x00 \x00\x80 \x00\x00\xa0\x00\x80\xa0\x00\x00 \x80\x80 \x80\x00\xa0\x80\x80\xa0\x80@ \x00\xc0 \x00@\xa0\x00\xc0\xa0\x00@ \x80\xc0 \x80@\xa0\x80\xc0\xa0\x80\x00`\x00\x80`\x00\x00\xe0\x00\x80\xe0\x00\x00`\x80\x80`\x80\x00\xe0\x80\x80\xe0\x80@`\x00\xc0`\x00@\xe0\x00\xc0\xe0\x00@`\x80\xc0`\x80@\xe0\x80\xc0\xe0\x80\x00 @\x80 @\x00\xa0@\x80\xa0@\x00 \xc0\x80 \xc0\x00\xa0\xc0\x80\xa0\xc0@ @\xc0 @@\xa0@\xc0\xa0@@ \xc0\xc0 \xc0@\xa0\xc0\xc0\xa0\xc0\x00`@\x80`@\x00\xe0@\x80\xe0@\x00`\xc0\x80`\xc0\x00\xe0\xc0\x80\xe0\xc0@`@\xc0`@@\xe0@\xc0\xe0@@`\xc0\xc0`\xc0@\xe0\xc0\xc0\xe0\xc0  \x00\xa0 \x00 \xa0\x00\xa0\xa0\x00  \x80\xa0 \x80 \xa0\x80\xa0\xa0\x80` \x00\xe0 \x00`\xa0\x00\xe0\xa0\x00` \x80\xe0 \x80`\xa0\x80\xe0\xa0\x80 `\x00\xa0`\x00 \xe0\x00\xa0\xe0\x00 `\x80\xa0`\x80 \xe0\x80\xa0\xe0\x80``\x00\xe0`\x00`\xe0\x00\xe0\xe0\x00``\x80\xe0`\x80`\xe0\x80\xe0\xe0\x80  @\xa0 @ \xa0@\xa0\xa0@  \xc0\xa0 \xc0 \xa0\xc0\xa0\xa0\xc0` @\xe0 @`\xa0@\xe0\xa0@` \xc0\xe0 \xc0`\xa0\xc0\xe0\xa0\xc0 `@\xa0`@ \xe0@\xa0\xe0@ `\xc0\xa0`\xc0 \xe0\xc0\xa0\xe0\xc0``@\xe0`@`\xe0@\xe0\xe0@``\xc0\xe0`\xc0`\xe0\xc0\xe0\xe0\xc0"

def load_ann_png(path):
    """Load a PNG file as a mask and its palette."""
    mask = Image.open(path)
    palette = mask.getpalette()
    mask = np.array(mask).astype(np.uint8)
    return mask, palette

def get_per_obj_mask(mask):
    """Split a mask into per-object masks."""
    object_ids = np.unique(mask)
    object_ids = object_ids[object_ids > 0].tolist()
    per_obj_mask = {object_id: (mask == object_id) for object_id in object_ids}
    return per_obj_mask

def put_per_obj_mask(per_obj_mask, height, width):
    """Combine per-object masks into a single mask."""
    mask = np.zeros((height, width), dtype=np.uint8)
    object_ids = sorted(per_obj_mask)[::-1]
    for object_id in object_ids:
        object_mask = per_obj_mask[object_id]
        object_mask = object_mask.reshape(height, width)
        mask[object_mask] = object_id
    return mask

def load_masks_from_dir(input_mask_path):
    input_mask, input_palette = load_ann_png(input_mask_path)
    per_obj_input_mask = get_per_obj_mask(input_mask)

    return per_obj_input_mask, input_palette

def save_predictions_to_dir(
    output_mask_dir,
    video_name,
    frame_name,
    per_obj_output_mask,
    height,
    width,
):
    """Save masks to a directory as PNG files."""
    os.makedirs(os.path.join(output_mask_dir, video_name), exist_ok=True)

    output_mask = put_per_obj_mask(per_obj_output_mask, height, width)
    output_mask_path = os.path.join(
        output_mask_dir, video_name, f"{frame_name}.png"
    )
    assert output_mask.dtype == np.uint8
    assert output_mask.ndim == 2
    output_mask = Image.fromarray(output_mask)
    output_mask.save(output_mask_path)

def create_overlay(img_path, mask_path, palette):
    """Create an overlay of an image and a mask."""
    img = Image.open(img_path)
    mask = Image.open(mask_path)
    mask.putpalette(palette)
    mask_rgb = mask.convert("RGB")
    mask_rgb = mask_rgb.resize(img.size, Image.NEAREST)
    overlay = Image.blend(img, mask_rgb, alpha=0.5)
    return overlay
    

In [6]:
# change to customized path
VIDEO_DIR = "/media/jma/MyPassport/MedSAM2/datasets/Demo/images"
VIDEO_NAME = "patient0213_4CH"
INITIAL_MASK_PROMPT = "/media/jma/MyPassport/MedSAM2/datasets/Demo/masks/patient0213_4CH/patient0213_4CH_half_sequence-000.png"
OUTPUT_DIR = "/media/jma/MyPassport/MedSAM2/datasets/Demo/output"

MODEL_CONFIG = "configs/sam2.1_hiera_t512.yaml"
MODEL_CHECKPOINT = "checkpoints/MedSAM2_latest.pt"

predictor = build_sam2_video_predictor(
    config_file=MODEL_CONFIG,
    ckpt_path=MODEL_CHECKPOINT,
    apply_postprocessing=True,
    # hydra_overrides_extra=hydra_overrides_extra,
    vos_optimized=  True,
)

## Prepare Inference and Add Initial Mask Prompt

In [3]:
# load the video frames
frame_names = [
        os.path.splitext(p)[0]
        for p in os.listdir(os.path.join(VIDEO_DIR, VIDEO_NAME))
        if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
    ]
frame_names = list(sorted(frame_names))
inference_state = predictor.init_state(
    video_path=os.path.join(VIDEO_DIR, VIDEO_NAME), async_loading_frames=False
)
height = inference_state["video_height"]
width = inference_state["video_width"]
input_palette = None

# Add input masks to MedSAM2 inference state before propagation
object_ids_set = None
input_frame_idx = 0  # use first frame as mask input
try:
    per_obj_input_mask, input_palette = load_masks_from_dir(input_mask_path=INITIAL_MASK_PROMPT)
except FileNotFoundError as e:
    raise RuntimeError(
        f"In {VIDEO_NAME=}, failed to load input mask for frame {input_frame_idx=}. "
        "Please add the `--track_object_appearing_later_in_video` flag "
        "for VOS datasets that don't have all objects to track appearing "
        "in the first frame (such as LVOS or YouTube-VOS)."
    ) from e

# get the list of object ids to track from the first input frame
if object_ids_set is None:
    object_ids_set = set(per_obj_input_mask)
for object_id, object_mask in per_obj_input_mask.items():
    # check and make sure no new object ids appear only in later frames
    if object_id not in object_ids_set:
        raise RuntimeError(
            f"In {VIDEO_NAME=}, got a new {object_id=} appearing only in a "
            f"later {input_frame_idx=} (but not appearing in the first frame). "
            "Please add the `--track_object_appearing_later_in_video` flag "
            "for VOS datasets that don't have all objects to track appearing "
            "in the first frame (such as LVOS or YouTube-VOS)."
        )
    predictor.add_new_mask(
        inference_state=inference_state,
        frame_idx=input_frame_idx,
        obj_id=object_id,
        mask=object_mask,
            )
    
# check and make sure we have at least one object to track
if object_ids_set is None or len(object_ids_set) == 0:
    raise RuntimeError(
        f"In {VIDEO_NAME=}, got no object ids on {input_frame_idx=}. "
        "Please add the `--track_object_appearing_later_in_video` flag "
        "for VOS datasets that don't have all objects to track appearing "
        "in the first frame (such as LVOS or YouTube-VOS)."
    )

FileNotFoundError: [Errno 2] No such file or directory: '/media/jma/MyPassport/MedSAM2/datasets/Demo/images/patient0213_4CH'

## Run Inference

In [5]:
# run propagation throughout the video
os.makedirs(os.path.join(OUTPUT_DIR, VIDEO_NAME), exist_ok=True)

video_segments = {}  # Store the per-frame segmentation results
for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(
    inference_state
):
    per_obj_output_mask = {
        out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }
    video_segments[out_frame_idx] = per_obj_output_mask

# write the output masks
for out_frame_idx, per_obj_output_mask in video_segments.items():
    # save raw prediction results
    save_predictions_to_dir(
        output_mask_dir=OUTPUT_DIR,
        video_name=VIDEO_NAME,
        frame_name=frame_names[out_frame_idx],
        per_obj_output_mask=per_obj_output_mask,
        height=height,
        width=width,
    )

PermissionError: [Errno 13] Permission denied: '/media/jma'

## Visualize Inference Results
Visualize segmentation results for 3 key frames at the 25%, 50%, and 75% position in the sequence

In [4]:
output_palette = input_palette or DAVIS_PALETTE
images = sorted(os.listdir(os.path.join(VIDEO_DIR, VIDEO_NAME)))
masks = sorted(os.listdir(os.path.join(OUTPUT_DIR, VIDEO_NAME)))
num_frames = len(images)
selected_indices = [int(num_frames * 0.25), int(num_frames * 0.5), int(num_frames * 0.75)]
selected_frames = [(images[i], masks[i]) for i in selected_indices]

plt.figure(figsize=(12, 4))
for idx, (img, mask) in enumerate(selected_frames):
    overlay = create_overlay(os.path.join(VIDEO_DIR, VIDEO_NAME, img), os.path.join(OUTPUT_DIR, VIDEO_NAME, mask), output_palette)

    plt.subplot(1, 3, idx + 1)
    plt.imshow(overlay)
    plt.title(f"Frame {int(num_frames*0.25*(idx+1))}")
    plt.axis("off")

plt.tight_layout()
plt.show()


NameError: name 'input_palette' is not defined